<a href="https://colab.research.google.com/github/surep83/mypythonsamples/blob/main/prompt_chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Workflow: Prompt chaining**

Prompt chaining decomposes a task into a sequence of steps, where each LLM call processes the output of the previous one.

You can add programmatic checks (see "gate” in the diagram below) on any intermediate steps to ensure that the process is still on track.


<img src="https://raw.githubusercontent.com/BrightPool/udemy-prompt-engineering-course/refs/heads/main/images/prompt-chaining.webp" alt="Agents" width="500">

## **Use cases:**

- Generating Marketing copy, then translating it into a different language.
- Writing an outline of a document, checking that the outline meets certain criteria, then writing the document based on the outline.
- Using an LLM to clean and standardize raw data, then passing the cleaned data to another LLM for insights, summaries, or visualizations.
- Generating a set of detailed questions based on a topic with one LLM, then passing those questions to another LLM to produce well-researched answers.


In [ ]:
%pip install openai pydantic --upgrade

In [ ]:
import os
from typing import List
from pydantic import BaseModel
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
class Story(BaseModel):
    story: str
    title: str
    author: str

class StoryPlot(BaseModel):
    plot: str

class StoryPlots(BaseModel):
    plots: List[StoryPlot]

In [ ]:
def serial_chain_workflow(topic: str, num_stories: int) -> List[Story]:
    """Run a serial chain of LLM calls to address the `input_query"""

    stories_plot_prompt = f'''Generate {num_stories} plot ideas for stories about {topic}.
    These should be short and concise, and should be suitable for a children's book.
    Later on these plot ideas will be used to generate full stories.'''

    stories_plot_response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[{"role": "user", "content": stories_plot_prompt}],
        temperature=0.5,
        max_tokens=1000,
        response_format=StoryPlots
    )

    if (stories_plot_response.choices[0].message.parsed.plots == []):
        raise ValueError("No plots generated")

    print("Plots:")
    print(stories_plot_response.choices[0].message.parsed.plots)

    # For each story plot, generate a full story
    stories = []
    for plot in stories_plot_response.choices[0].message.parsed.plots:
        story_prompt = f'''Generate a full story based on the following plot: {plot.plot}.
        The story should be suitable for a children's book. The title should not be within the story part of the response.
        If there is no author, then write "Unknown"'''

        story_response = client.beta.chat.completions.parse(
            model="gpt-4o",
            messages=[{"role": "user", "content": story_prompt}],
            temperature=0.5,
            max_tokens=1000,
            response_format=Story
        )

        if (story_response.choices[0].message.parsed.story == ""):
            raise ValueError("No story generated")

        stories.append(story_response.choices[0].message.parsed)
    return stories

In [ ]:
stories = serial_chain_workflow("data science", 2)

Plots:
[StoryPlot(plot='**Title: The Magical Data Garden**  \n**Plot:** In the vibrant town of Numberville, young Ada discovers a secret garden where data grows on trees! Each tree represents different types of data: numbers, shapes, colors, and even sounds. Ada befriends a wise old owl named Professor Hoot, who teaches her how to harvest and use the data to solve puzzles and help her friends. Together, they embark on a journey to unlock the mysteries of the garden, learning the importance of patterns, predictions, and sharing knowledge along the way. The story emphasizes teamwork and curiosity as Ada and her friends use data to make their world a better place.'), StoryPlot(plot="**Title: Benny's Big Data Adventure**  \n**Plot:** Benny the Bear loves playing games on his tablet, but one day, all his games stop working! With the help of Dot, a friendly digital sprite, Benny learns that his games run on data, and it's up to him to fix it. Dot leads Benny on a thrilling adventure through 

In [ ]:
print(stories[0].title)
print(stories[0].author)

The Magical Data Adventure
[Your Name]
